# Train Models Using LeRobot on MI300x

This guide walks you through setting up environment for training imitation learning policies using LeRobot library on a DigitalOcean (DO) instance equipped with AMD MI300x GPUs and ROCm.

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=${HF_USER}/${DATASET_NAME}`).
  If you don’t have an access token yet, you can sign up for Hugging Face [here](https://huggingface.co/join). After signing up, create an access token by visiting [here](https://huggingface.co/settings/tokens).
- A wandb account to enable training visualization and upload your training evidence to our github.
  You can sign up for Wandb [here](https://wandb.ai/signup) and visit [here](https://wandb.ai/authorize) to create a token.
- Access to DO instance AMD Mi300x GPU


## Verify ROCm and GPU availability
This cell uses `pytorch` to check AMD GPU Info. The expected ouput is 
```
CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF
```

In [1]:
import torch
print(f'CUDA compatible device availability:',torch.cuda.is_available())
print(f'device name [0]:', torch.cuda.get_device_name(0))


CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF


## Install FFmpeg 7.x
This cell uses `apt` to install ffmpeg 7.x for LeRobot.

In [2]:
!add-apt-repository ppa:ubuntuhandbook1/ffmpeg7 -y # install PPA which contains ffmpeg 7.x
!apt update && apt install ffmpeg -y

Repository: 'Types: deb
URIs: https://ppa.launchpadcontent.net/ubuntuhandbook1/ffmpeg7/ubuntu/
Suites: noble
Components: main
'
Description:
unofficial build for FFmpeg 7 for Ubuntu 22.04 | 24.04, backport from Debian's deb.multimedia.org repository

If the packages here are helpful, you may buy me a coffee:

         https://ko-fi.com/ubuntuhandbook1
More info: https://launchpad.net/~ubuntuhandbook1/+archive/ubuntu/ffmpeg7
Adding repository.
Hit:1 https://repo.radeon.com/amdgpu/30.10/ubuntu jammy InRelease
Hit:2 https://repo.radeon.com/rocm/apt/7.0 jammy InRelease                     
Hit:3 https://repo.radeon.com/graphics/7.0/ubuntu jammy InRelease              
Get:4 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu noble InRelease                         
Get:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu noble InRelease [17.8 kB]


## Install LeRobot v0.4.1
This cell clones the `lerobot` repository from Hugging Face, and installs the package in editable mode. Extra Features: To install additional dependencies for training SmolVLA or Pi models, refer to the [LeRobot offical page](https://huggingface.co/docs/lerobot/index). 


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!cd lerobot && git checkout -b v0.4.1 v0.4.1 # let’s synchronize using this version
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 44940, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 44940 (delta 79), reused 83 (delta 63), pack-reused 44807 (from 2)
Receiving objects: 100% (44940/44940), 221.03 MiB | 67.36 MiB/s, done.
Resolving deltas: 100% (29041/29041), done.
Switched to a new branch 'v0.4.1'
Obtaining file:///workspace/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 86.1 MB/s  0:00:00 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 115.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/

## Weights & Biases login
This cell install and log into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
!pip install wandb
import wandb
wandb.login(key="wandb_key")

## Login into Hugging Face Hub

In [ ]:
from huggingface_hub import login
login(token="hf_token")

## Start Training Models with LeRobot

This cell uses the lerobot-train CLI from the lerobot library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `lerobot/svla_so100_pickplace`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.The name typically includes the model type (e.g., act, smolvla), the dataset name, and additional descriptive tags.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an AMD or NVIDIA GPU. 

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

7. `--policy.push_to_hub=`:

   Enables automatic uploading of the trained policy to the Hugging Face Hub. You must specify `--policy.repo_id` (e.g., ${HF_USER}/{REPO_NAME}) if it is True.

In [31]:
!lerobot-train \
  --dataset.repo_id=giacomoran/hackathon_amd_mission1 \
  --batch_size=64 \
  --steps=5000 \
  --output_dir=outputs/train/hackathon_amd_mission1 \
  --job_name=hackathon_amd_mission1 \
  --policy.repo_id=giacomoran/hackathon_amd_mission1 \
  --policy.device=cuda \
  --policy.type=act \
  --policy.push_to_hub=true \
  --wandb.enable=true \
  --policy.chunk_size=30 \
  --policy.n_action_steps=30 \
  --optimizer.lr=5e-5 \
  --policy.optimizer_lr=5e-5 \
  --policy.optimizer_lr_backbone=5e-5 \
  --save_freq=2000

INFO 2025-12-12 21:34:10 ot_train.py:163 {'batch_size': 64,
 'checkpoint_path': None,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'affine': {'kwargs': {'degrees': [-5.0,
                                                                            5.0],
                                                                'translate': [0.05,
                                                                              0.05]},
                                                     'type': 'RandomAffine',
                                                     'weight': 1.0},
                                          'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         '

**Notes**:

- If using a local dataset, add `--dataset.root=/path/to/dataset`.
- Adjust `--batch_size` and `--steps` based on your hardware and dataset.
- Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`
- Training progress visualized in your wandb dashboard


## Download Models from Hugging Face to Local Machine
Now after training is done, download the model to local machine. 

In [32]:
!huggingface-cli download giacomoran/hackathon_amd_mission1 --repo-type model --local-dir outputs/train/hackathon_amd_mission1
# e.g. huggingface-cli upload ${HF_USER}/act_so101_3cube_1ksteps \
#  outputs/train/act_so101_3cube_1ksteps/checkpoints/last/pretrained_model

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 9 files:   0%|                                   | 0/9 [00:00<?, ?it/s]Downloading 'policy_preprocessor_step_3_normalizer_processor.safetensors' to 'outputs/train/hackathon_amd_mission1/.cache/huggingface/download/KTJX-ilfk9YZ9PyHew1aNVsbRpk=.75bebdcbfc0f385e5ac9989dbe6cc1d61b17283609bb5ad4a05911a6e6ad4727.incomplete'

policy_preprocessor_step_3_normalizer_pr(…):   0%|  | 0.00/7.54k [00:00<?, ?B/s]

config.json: 1.67kB [00:00, 12.3MB/s]A
Download complete. Moving file to outputs/train/hackathon_amd_mission1/config.json


policy_postprocessor.json: 100%|███████████████| 660/660 [00:00<00:00, 9.26MB/s]
Download complete. Moving file to outputs/train/hackathon_amd_mission1/policy_postprocessor.json


policy_preprocessor.json: 1.33kB [00:00, 3.77MB/s]A
Download complete. Moving file to outputs/train/hackathon_amd_mission1/policy_preprocessor.json


policy_postprocessor_step_0_unnormalizer(…):   0%|  

## Miscs
1. Once the environment is setup, you can open a terminal session for training by navigating to `File → New Launcher → Other → Terminal`.
2. You can also upload your datasets to the container by clicking the `Upload Files` button in the left pane.

## Q&A
1. If you encounter an error like:
   ```
   FileExistsError: Output directory outputs/train/act_so101_3cube_1ksteps already exists and resume is False. Please change your output directory so that outputs/train/act_so101_3cube_1ksteps is not overwritten. 
   ```
   Remove the existing directory before proceeding:

In [ ]:
!rm -fr outputs/train/hackathon_amd_mission1

2. When running models other than ACT, ensure you install the required additional dependencies for those models.

In [ ]:
# For smolVLA
!cd lerobot && pip install -e ".[smolvla]"
# For Pi
!cd lerobot && pip install -e ".[pi]"

3. If you want to resume the training from last checkpoint, run the command below:

In [ ]:
!lerobot-train \
  --resume=true \
  --config_path=outputs/train/<job name>/checkpoints/last/pretrained_model/train_config.json \
  --steps=<new total steps>

4. If you want to upload your dataset using `huggingface-cli upload <repo name> <path to the dataset> --repo-type=dataset`, be sure to set a codebase tag like below:

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import login

login(token="your_huggingface_token")
hub_api = HfApi()
hub_api.create_tag(<HF_REPO_NAME>, tag="v3.0", revision="main", repo_type="dataset")